In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd
import pickle

# Physics-related Packages
from astropy.cosmology import Planck15 as cosmo

In [ ]:
# preparations for read box info from the url
import requests
import time

baseUrl = 'http://www.tng-project.org/api/'

def get(path, params=None, max_retries=5, backoff_factor=2):
    # make HTTP GET request to path
    headers = {"api-key":"API KEY"}

    attempt = 0
    while attempt < max_retries:
        try:
            r = requests.get(path, params=params, headers=headers)
            r.raise_for_status()

            if r.headers['content-type'] == 'application/json':
                return r.json()
            
            if 'content-disposition' in r.headers:
                filename = r.headers['content-disposition'].split("filename=")[1]
                with open(filename, 'wb') as f:
                    f.write(r.content)
                return filename
            return r  # fallback
        
        except Exception as e:
            attempt += 1
            wait = backoff_factor ** attempt
            print(f"[Retry {attempt}/{max_retries}] Request failed: {e}. Retrying in {wait}s...")
            time.sleep(wait)

    raise RuntimeError(f"Failed to GET {path} after {max_retries} retries.")

# Issue a request to the API root
r = get(baseUrl)

# Print out all the simulation names
names = [sim['name'] for sim in r['simulations']]
# Get the index of TNG300-1
i = names.index('TNG-Cluster')
# Get the info of simulation Illustris-3
sim = get( r['simulations'][i]['url'] )
sim.keys()

# get the snaps info this simulation
snaps = get(sim['snapshots'])

# Sim Box parameters
Snap_Index = 99 # the snapshots index in the total 100 snapshots taking at different z
BoxSize = sim['boxsize'] # unit: ckpc/h
Redshift = snaps[Snap_Index]['redshift'] # current redshift of our current snap

# Short description of cosmological parameters Planck2015
h_atz = cosmo.H(Redshift).value/100 # unit 100km/[s*Mpc]

LowerMass_lim = 10**3 * h_atz # units 10^ 10 𝑀⊙/ℎ * h
HigherMass_lim = 10**8 * h_atz # units 10^ 10 𝑀⊙/ℎ * h

In [ ]:
base_cat_name = '/users_path/merger_trace/tng_cluster/tng_cluster_targetcat/'

mpb_base_path = '/users_path/merger_trace/tng_cluster/tng_cluster_mpbs/'

In [ ]:
def get_subhalo_maxM(Halo_ID, Sub_GrNr, Sub_Mass):
    
    """
    Finds the indices of the top three most massive subhalos for a given halo.

    Parameters:
    - Halo_ID: Array of halo IDs (single value or array with specific halo ID to match).
    - Sub_GrNr: Array indicating the group number (halo) each subhalo belongs to.
    - Sub_Mass: Array of subhalo masses.

    Returns:
    - Numpy array (CenterSub_Index, SecondSub_Index, ThirdSub_Index):
      - CenterSub_Index: Index of the most massive subhalo in the halo.
      - SecondSub_Index: Index of the second most massive subhalo.
      - ThirdSub_Index: Index of the third most massive subhalo.
    """
        
    find_Sub = np.where(Sub_GrNr == Halo_ID)[0]
    find_Sub_Mass = Sub_Mass[find_Sub]
    find_Sub_Mass_Sorted = np.argsort(find_Sub_Mass)

    CenterSub_Index = np.where(Sub_GrNr == Halo_ID)[0][find_Sub_Mass_Sorted[-1]]

    CenterSub_Index = np.array(CenterSub_Index)
    
    return CenterSub_Index

def Get_AvgSFR(SubhaloGrNr, SubhaloSFR, FOF_Halo_IDs):
    AvgSFR = np.zeros(FOF_Halo_IDs.shape)

    for i in range(len(FOF_Halo_IDs)):
        current_fof_id = FOF_Halo_IDs[i]
        sub_in_fof = (SubhaloGrNr == current_fof_id)
        subSFR_in_fof = SubhaloSFR[sub_in_fof]
        AvgSFR[i] = np.mean(subSFR_in_fof)

    return AvgSFR
    
def Get_HaloIDs(TargetHalo_cat):
    """
    Extracts halo IDs and their properties from the HDF5 catalog.

    Parameters:
    - TargetHalo_cat: Path to the HDF5 file containing the target halo catalog.

    Returns:
    - Target_Halo_IDs: List of selected halo IDs.
    - Subhalo_MaxMasses: Maximum subhalo masses for each selected halo.
    - Target_Halo_Rs_Crit200: Halo critical radius R_Crit200.
    - Target_GroupPoses: Positions of the selected halos.
    - Galaxy_nums: Number of subhalos per halo.
    """

    with h5py.File(TargetHalo_cat, 'r') as Target_hdf:
        # Read FOF Halo Info
        FOF_Halo_IDs =  Target_hdf['Group/FOF_Halo_IDs'][:]
        GroupFirstSub = Target_hdf['Group/GroupFirstSub'][:]
        Group_Nsubs = Target_hdf['Group/GroupNsubs'][:]

        # Read Subhalo Info
        SubhaloGrNr = Target_hdf['Subhalo/SubhaloGrNr'][:]
        SubhaloSFR = Target_hdf['Subhalo/SubhaloSFR'][:]
        SubhaloIDs = Target_hdf['Subhalo/Subhalo_IDs'][:]

    # Find FOF halos with subhalos and Read Info
    Indices_HaloWithSub = np.where( GroupFirstSub != -1)[0]
    Target_Halo_IDs = FOF_Halo_IDs[Indices_HaloWithSub]

    # Initialize array to get the central subhalo, second massive subhalo, and the third massive subhalo

    # Initialize galaxy numbers
    Galaxy_nums = Group_Nsubs[np.isin(FOF_Halo_IDs, Target_Halo_IDs)]
    AvgSFR = Get_AvgSFR(SubhaloGrNr, SubhaloSFR, FOF_Halo_IDs)

    return (Target_Halo_IDs, Galaxy_nums,AvgSFR, SubhaloGrNr, SubhaloIDs)

In [ ]:
with h5py.File('/users_path/data/tng_cluster/tng_cluster_catalog/TNG-Cluster_Catalog.hdf5', 'r') as f:
    origID_parentsim = f['origID'][:]
    haloID__TNGCluster = f['haloID'][:]

cat_name ='/users_path/data/tng_cluster/tng_cluster_targetcat/targethalo_cat_099/TargetHalo_MergerCat_099.hdf5'

results = Get_HaloIDs(cat_name)

(Target_Halo_IDs, _,  _, _, _) = results


with h5py.File('/users_path/data/tng_cluster/tng_cluster_cluster_mergers/cluster_mergers.hdf5', 'r') as f:
    HaloID = f['HaloID'][:]
    Snap_coll = f['Snap_coll'][:]
    print(f.keys())


HaloID_Mergercat = np.array(list(set(HaloID))).astype(int)


Merger_Target_Halo_ID = range(len(haloID__TNGCluster))
Merger_Target_Halo_ID = np.array(Merger_Target_Halo_ID)

Snap_coll_eachhalo = dict()

for i in range(len(Merger_Target_Halo_ID)):
    current_halo_id = Merger_Target_Halo_ID[i]
    rel_merger_snap = Snap_coll[HaloID==current_halo_id]
    halo_id_frommergercat2TNGCluster = haloID__TNGCluster[current_halo_id] 
    Snap_coll_eachhalo[halo_id_frommergercat2TNGCluster] = rel_merger_snap

In [ ]:
def get_snaptime(snap):
    snap_redshift = snaps[snap]['redshift'] 
    t_cosmic = cosmo.age(snap_redshift).value  # age of the Universe at that redshift
    return t_cosmic

In [ ]:
def get_apparent_mag(abs_mag, redshift_snap):
    D_L = cosmo.luminosity_distance(redshift_snap).to('pc').value
    apparent_mag = abs_mag + 5 * np.log10(D_L / 10)

    return apparent_mag


In [ ]:
def get_bluefrac(halo_id_snap, cat_name_snap, redshift_snap):
    with h5py.File(cat_name_snap, 'r') as f:
        FOF_Halo_IDs =  f['Group/FOF_Halo_IDs'][:]
        Group_Nsubs =f['Group/GroupNsubs'][:]
        SubhaloGrNr = f['Subhalo/SubhaloGrNr'][:]
        SubhaloStellarPhotometrics = f['Subhalo/SubhaloStellarPhotometrics'][:] # U, B, V, K, g, r, i, z. 
        
    subhalo_idx = np.where(SubhaloGrNr==halo_id_snap)[0]
    assert Group_Nsubs[FOF_Halo_IDs==halo_id_snap] == len(subhalo_idx)
    U_band_data = SubhaloStellarPhotometrics[subhalo_idx, 0]
    #V_band_data = SubhaloStellarPhotometrics[subhalo_idx, 2]
    g_band_data = SubhaloStellarPhotometrics[subhalo_idx, 4]
    # make magnitude cut
    apparent_U_mag = get_apparent_mag(U_band_data, redshift_snap)
    #apparent_V_mag = get_apparent_mag(V_band_data, redshift_snap)
    apparent_g_mag = get_apparent_mag(g_band_data, redshift_snap)
    #mask = np.where((apparent_U_mag<=22) & (apparent_V_mag<=22))[0]
    mask = np.where((apparent_U_mag<=23) & (apparent_g_mag<=25))[0]
    visible_number = len(mask)
    U_band_masked = U_band_data[mask]
    # V_band_masked = V_band_data[mask]
    g_band_masked = g_band_data[mask]
    blue_galaxy_number_05 = np.sum((U_band_masked - g_band_masked)<=0.5)
    blue_galaxy_number_03 = np.sum((U_band_masked - g_band_masked )<=0.3)


    blue_frac_05 = blue_galaxy_number_05/visible_number
    blue_frac_03 = blue_galaxy_number_03/visible_number

    print(visible_number)

    return blue_frac_05, blue_frac_03


In [ ]:
def get_scores_fromdict(halo_id_99, center_snap, features_dict):

    halo_data = features_dict[halo_id_99][center_snap]
    if 'label_score_all_tau2.0' not in halo_data  or 'label_score_pre_tau2.0' not in halo_data:
        return None, None  

    all_merger_score = halo_data['label_score_all_tau2.0']

    pre_merger_score = halo_data['label_score_pre_tau2.0']

    post_merger_score = all_merger_score-pre_merger_score

    return pre_merger_score, post_merger_score

In [ ]:
with open('/users_path/merger_trace/data/tng_cluster/tng_cluster_products/feats_labels_dict_tngcluster.pkl', 'rb') as f:
    features_dict = pickle.load(f)

In [ ]:
MergerHaloID_TNGCluster = np.array([haloID__TNGCluster[i] for i in HaloID]).astype(int)

In [ ]:
cat_name_99 = base_cat_name + 'targethalo_cat_099/TargetHalo_MergerCat_099.hdf5'

Target_Halo_IDs_99, Galaxy_nums_99, AvgSFR_99, SubhaloGrNr_99, SubhaloIDs_99= Get_HaloIDs(cat_name_99)

rows = [] 
for snap in range(72, 100):
    redshift_snap = snaps[snap]['redshift']
    if redshift_snap<=0.002:
        redshift_snap = 0.0025
    for fof_halo_id_99 in Snap_coll_eachhalo.keys():
        sub1st_at99 = int(SubhaloIDs_99[SubhaloGrNr_99==fof_halo_id_99][0])

        with h5py.File(mpb_base_path+f'sublink_mpb_{sub1st_at99}.hdf5', 'r') as mpb_f:
            SubfindID = mpb_f['SubfindID'][:]
            SnapNum = mpb_f['SnapNum'][:]

        cat_name_snap = base_cat_name + f'targethalo_cat_0{snap}/TargetHalo_MergerCat_0{snap}.hdf5'

        Target_Halo_IDs_snap, Galaxy_nums_snap, AvgSFR_snap, SubhaloGrNr_snap, SubhaloIDs_snap= Get_HaloIDs(cat_name_snap)
        
        progenitor_id_snap = SubfindID[SnapNum==snap]

        if len(progenitor_id_snap)==0:
            continue

        halo_id_snap = int(SubhaloGrNr_snap[SubhaloIDs_snap==progenitor_id_snap][0])
     
        blue_frac_05, blue_frac_03 = get_bluefrac(halo_id_snap, cat_name_snap, redshift_snap)

        pre_merger_score, post_merger_score = get_scores_fromdict(fof_halo_id_99, snap, features_dict)
        print(pre_merger_score, post_merger_score)
        meta_info = (fof_halo_id_99, snap)

        rows.append({
            "fof_halo_id_99": fof_halo_id_99,
            "snap": snap,
            "halo_id_snap": halo_id_snap,
            "blue_frac_05": blue_frac_05,
            "blue_frac_03": blue_frac_03,
            "merger_score_pre": pre_merger_score,
            "merger_score_post": post_merger_score
        })

        print(f'finish {fof_halo_id_99} at snap {snap} where it was {halo_id_snap} with {blue_frac_03}')


In [ ]:
# save to csv file
df = pd.DataFrame(rows)
df.to_csv("blue_frac_merger_score_ug.csv", index=False)

## pre merger
    
    
plot the correlation between the pre-merger score and blue galaxy fraction.

In [ ]:

df = pd.read_csv("/home/chuiyang/merger_trace/notebooks/analysis/blue_frac_merger_score_ug.csv")

pre_merger_score_vec = df['merger_score_pre'].values
redshift_vec = df['snap'].values
blue_frac_vec = df['blue_frac_03'].values
post_merger_score_vec = df['merger_score_post'].values

mask = (redshift_vec<=99)

pre_merger_score_vec = pre_merger_score_vec[mask]
redshift_vec = redshift_vec[mask]
blue_frac_vec = blue_frac_vec[mask]
post_merger_score_vec = post_merger_score_vec[mask]


In [ ]:
z_bins = np.array([72., 77., 82., 87., 93., 100.])# 5 bins for redshift
score_bins = np.percentile(pre_merger_score_vec, [0, 20, 40, 60, 80, 100])


# created 2D heatmap
heatmap = np.full((len(z_bins)-1, len(score_bins)-1), np.nan)
# add std matrix
std_map = np.full((len(z_bins) - 1, len(score_bins) - 1), np.nan)
std_N_map = np.full((len(z_bins) - 1, len(score_bins) - 1), np.nan)
sample_count = np.zeros_like(heatmap, dtype=int)

for i in range(len(z_bins) - 1):
    for j in range(len(score_bins) - 1):
        mask = (
            (redshift_vec >= z_bins[i]) & (redshift_vec < z_bins[i+1]) &
            (pre_merger_score_vec >= score_bins[j]) & (pre_merger_score_vec < score_bins[j+1])
        )
        values = blue_frac_vec[mask]
        sample_count[i, j] = np.sum(mask)
        if sample_count[i, j] > 0:
            heatmap[i, j] = np.mean(values)
            std_map[i, j] = np.std(values)
            std_N_map[i, j] = np.std(values)/np.sqrt(len(values))

In [ ]:

score_centers = [(score_bins[i] + score_bins[i+1]) / 2 for i in range(len(score_bins)-1)]

colors = plt.cm.viridis(np.linspace(0, 1, len(z_bins)-1))

plt.figure(figsize=(8, 6))

for i in range(len(z_bins) - 1):
    mean = heatmap[i]
    std = std_N_map[i]
    label = f'{z_bins[i]:.0f}-{z_bins[i+1]:.0f} z'
    
    plt.errorbar(score_centers, mean, yerr=std, label=label,
                 fmt='-o', capsize=4, color=colors[i])

plt.xlabel('Pre-merger Score')
plt.ylabel('Blue Fraction')
plt.title('Blue Fraction vs. Pre-merger Score (with std error bars)')
plt.legend(title='Redshift Bins')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
z_labels = [f'{z_bins[i]:.0f}-{z_bins[i+1]:.0f} z' for i in range(len(z_bins)-1)]
score_labels = list(score_centers)


data = []
for i in range(len(z_bins)-1):
    for j in range(len(score_bins)-1):
        data.append({
            'z_bin': z_labels[i],
            'score_bin': score_labels[j],
            'mean': heatmap[i, j],
            'std_error': std_N_map[i, j],
            'sample_count': sample_count[i, j]
        })

# save data for plots
df = pd.DataFrame(data)

df.to_csv("blue_frac_plot_data_pre_ug.csv", index=False)



## post megrer

plot the correlation between the post-merger score and blue galaxy fraction.

In [ ]:
df = pd.read_csv("/home/chuiyang/merger_trace/notebooks/analysis/blue_frac_merger_score_ug.csv")

redshift_vec_all = df['snap'].values
blue_frac_vec_all = df['blue_frac_03'].values
post_merger_score_vec_all = df['merger_score_post'].values

# cut snapshots near redshift 0 since they lack future merger information
snap_cut = 87
mask = redshift_vec_all<=87

redshift_vec = redshift_vec_all[mask]
blue_frac_vec = blue_frac_vec_all[mask]
post_merger_score_vec = post_merger_score_vec_all[mask]

In [ ]:
z_bins = np.linspace(min(redshift_vec), max(redshift_vec), 4)  # 5 bins for redshift
score_bins = np.percentile(post_merger_score_vec, [0, 20, 40, 60, 80, 100])

heatmap = np.full((len(z_bins)-1, len(score_bins)-1), np.nan)

std_map = np.full((len(z_bins) - 1, len(score_bins) - 1), np.nan)
std_N_map = np.full((len(z_bins) - 1, len(score_bins) - 1), np.nan)
sample_count = np.zeros_like(heatmap, dtype=int)

for i in range(len(z_bins) - 1):
    for j in range(len(score_bins) - 1):
        mask = (
            (redshift_vec >= z_bins[i]) & (redshift_vec < z_bins[i+1]) &
            (post_merger_score_vec >= score_bins[j]) & (post_merger_score_vec < score_bins[j+1])
        )
        values = blue_frac_vec[mask]
        sample_count[i, j] = np.sum(mask)
        if sample_count[i, j] > 0:
            heatmap[i, j] = np.mean(values)
            std_map[i, j] = np.std(values)
            std_N_map[i, j] = np.std(values)/np.sqrt(len(values))

In [ ]:
score_centers = [(score_bins[i] + score_bins[i+1]) / 2 for i in range(len(score_bins)-1)]

colors = plt.cm.viridis(np.linspace(0, 1, len(z_bins)-1))

plt.figure(figsize=(8, 6))

for i in range(len(z_bins) - 1):
    mean = heatmap[i]
    std = std_N_map[i]
    label = f'{z_bins[i]:.0f}-{z_bins[i+1]:.0f} z'
    
    plt.errorbar(score_centers, mean, yerr=std, label=label,
                 fmt='-o', capsize=4, color=colors[i])

plt.xlabel('Pre-merger Score')
plt.ylabel('Blue Fraction')
plt.title('Blue Fraction vs. Post-merger Score (with std error bars)')
plt.legend(title='Redshift Bins')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
z_labels = [f'{z_bins[i]:.0f}-{z_bins[i+1]:.0f} z' for i in range(len(z_bins)-1)]
score_labels = list(score_centers)

data = []
for i in range(len(z_bins)-1):
    for j in range(len(score_bins)-1):
        data.append({
            'z_bin': z_labels[i],
            'score_bin': score_labels[j],
            'mean': heatmap[i, j],
            'std_error': std_N_map[i, j],
            'sample_count': sample_count[i, j]
        })

df = pd.DataFrame(data)

df.to_csv("blue_frac_plot_data_post_ug.csv", index=False)
